# Testing notebook

This notebook is used to produce the testing by the tools. 

The goal here is simply to call the APIs in question and record their answers on our datasets. No analysis is conducted here, this is purely to get their answers for later. 

Here, we test the following tools : 
- Namsor
- GenderAPI.io
- GenderAPI.com
- Genderize
- gender_guesser

The last one, gender_guesser, is not an API, so it is not tested along the rest of the other tools, but is done in its own cells. 

Once the results of all those tools are defined, the results are stored as `.csv` file for analysis later. 

In [2]:
import pandas as pd
%cd ../..
from notebooks.helpers.serviceWrapperAsync import NamSorEndpoint 
from notebooks.helpers.serviceHandlers import GenderGuesserHandler
from notebooks.helpers.wrapperManager import WrapperManager
import notebooks.helpers.serviceWrapperAsync as wrapper
import asyncio
import random
%load_ext autoreload
%autoreload 2

c:\Users\brief\Desktop\Master\Ecole\Semestre3


C:\Users\brief\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
master_df = pd.read_csv('C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/all.csv')

In [28]:
master_df.head()

,id,fullName,firstName,lastName,gender,isoCountry,continent,birthYear,hasMiddleName,hasNoLastName,source
0,0,Clara Benson,Clara,Benson,female,GH,Africa,2000,False,False,wikidata
1,1,Esther Ruth Mbabazi,NaN,NaN,female,UG,Africa,1995,True,False,wikidata
2,2,Dalia Ziada,Dalia,Ziada,female,EG,Africa,1982,False,False,wikidata
3,3,Fatou Haidara,Fatou,Haidara,female,ML,Africa,1962,False,False,wikidata
4,4,Claude Haffner,Claude,Haffner,female,NaN,NaN,1976,False,False,wikidata


## Useful batch method to avoid problems rate limit and load balancer

In [ ]:
async def run_batched_wrapper(
    wrapper_class,              # e.g. wrapper.GenderAPI_com_Wrapper
    df: pd.DataFrame,  # dataset
    batch_size: int = 100,      # number of rows per batch
    pause_seconds: float = 60,  # base cooldown between batches
    **kwargs                    # passed directly to get_prediction_async()
) -> pd.DataFrame:
    """
    Run any gender inference wrapper (GenderAPI, NamSor, Genderize, etc.)
    in batched mode with adaptive cooldown handling.

    Parameters
    ----------
    wrapper_class : class
        The wrapper class (not instance) from `serviceWrapperAsync`, e.g. wrapper.NamSorWrapper.
    df : pd.DataFrame
        The dataset to process.
    batch_size : int
        Number of rows per batch.
    pause_seconds : float
        Base number of seconds to wait between batches.
    **kwargs :
        Additional arguments passed to `get_prediction_async()`
        (e.g. useLocalization=True, endpoint=NamSorEndpoint.FIRST_NAME_GEO).

    Returns
    -------
    combined_df : pd.DataFrame
        Combined DataFrame containing all parsed batch results.
    """

    results = []
    total = len(df)
    

    for start in range(0, total, batch_size):
        end = min(start + batch_size, total)
        print(f"\nProcessing rows {start}-{end}...")

        # Prepare the batch DataFrame
        batch_df = df.iloc[start:end]

        # Instantiate wrapper for this batch
        wrap_instance = wrapper_class(batch_df)

        # Run async API calls
        batch_result = await wrap_instance.get_prediction_async(**kwargs)
        results.append(batch_result)

        # Pause between batches
        if end < total:
            
            sleep_time = pause_seconds + random.uniform(1, 5)  # it's common for API limit to be over precisely 1 minute 
            # or other whole time limit. By adding a random, little amount of time to each pause, we should more easily 
            # squirt this problem.  
            print(f"Sleeping {sleep_time:.1f}s to respect API limits...")
            await asyncio.sleep(sleep_time)

    # Merge all batch results
    combined_df = pd.concat(results, ignore_index=True)
    print("\n All batches completed successfully.")
    return combined_df


## Wikidata first name + localization

In [ ]:
wikidata_First_Country = master_df[
    (master_df['source'] == 'wikidata') &
    (master_df['firstName'].notnull()) &
    (master_df['isoCountry'].notnull())
]
wikidata_First_Country.shape

### Genderize

In [132]:
result_Wikidata_First_Country_Genderize = await wrapper.GenderizeWrapper(wikidata_First_Country).get_prediction_async(useLocalization=True)
result_Wikidata_First_Country_Genderize

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
0,0,Clara Benson,Clara,female,female,GH,True,genderize.IO,247083,0.99
1,2,Dalia Ziada,Dalia,female,female,EG,True,genderize.IO,87214,0.99
2,3,Fatou Haidara,Fatou,female,female,ML,True,genderize.IO,25477,0.98
3,7,Diallo Sène,Diallo,female,male,ML,True,genderize.IO,8032,0.82
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderize.IO,15498,0.99
...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,genderize.IO,324,1.00
3641,4865,Sir Chandler,Sir,male,male,AR,True,genderize.IO,3124,0.96
3642,4866,Julio Fierro,Julio,male,male,CL,True,genderize.IO,478589,1.00
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,genderize.IO,3,1.00


In [139]:
result_Wikidata_First_Country_Genderize.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Genderize/wikidataFirstCountry.csv", index=False)

### GenderAPI.io

In [140]:
result_Wikidata_First_Country_GenderAPIIO = await wrapper.GenderAPI_IO_Wrapper(wikidata_First_Country).get_prediction_async(useLocalization=True)

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
0,0,Clara Benson,Clara,female,female,GH,True,genderize.IO,247083,0.99
1,2,Dalia Ziada,Dalia,female,female,EG,True,genderize.IO,87214,0.99
2,3,Fatou Haidara,Fatou,female,female,ML,True,genderize.IO,25477,0.98
3,7,Diallo Sène,Diallo,female,male,ML,True,genderize.IO,8032,0.82
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderize.IO,15498,0.99
...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,genderize.IO,324,1.00
3641,4865,Sir Chandler,Sir,male,male,AR,True,genderize.IO,3124,0.96
3642,4866,Julio Fierro,Julio,male,male,CL,True,genderize.IO,478589,1.00
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,genderize.IO,3,1.00


In [145]:
result_Wikidata_First_Country_GenderAPIIO[result_Wikidata_First_Country_GenderAPIIO['predictedGender']=='null'] = 'unknown'
result_Wikidata_First_Country_GenderAPIIO['predictedGender'].value_counts()

predictedGender
male       1794
female     1731
unknown     120
Name: count, dtype: int64

In [146]:
result_Wikidata_First_Country_GenderAPIIO.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIIO/wikidataFirstCountry.csv", index=False)

### GenderAPI.com

In [21]:
result_Wikidata_First_Country_GenderAPIcom = await run_batched_wrapper(
    wrapper.GenderAPI_com_Wrapper,
    wikidata_First_Country,
    batch_size=50,
    pause_seconds=10,
    useLocalization=True,
    useFullName=False
)
result_Wikidata_First_Country_GenderAPIcom


Processing rows 0-50...
Sleeping 12.7s to respect API limits...

Processing rows 50-100...
Sleeping 11.5s to respect API limits...

Processing rows 100-150...
Sleeping 13.3s to respect API limits...

Processing rows 150-200...
Sleeping 11.1s to respect API limits...

Processing rows 200-250...
Sleeping 14.3s to respect API limits...

Processing rows 250-300...
Sleeping 13.4s to respect API limits...

Processing rows 300-350...
Sleeping 12.7s to respect API limits...

Processing rows 350-400...
Sleeping 11.3s to respect API limits...

Processing rows 400-450...
Sleeping 12.6s to respect API limits...

Processing rows 450-500...
Sleeping 14.7s to respect API limits...

Processing rows 500-550...
Sleeping 14.0s to respect API limits...

Processing rows 550-600...
Sleeping 13.6s to respect API limits...

Processing rows 600-650...
Sleeping 14.5s to respect API limits...

Processing rows 650-700...
Sleeping 13.2s to respect API limits...

Processing rows 700-750...
Sleeping 14.7s to respec

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
0,0,Clara Benson,Clara,female,female,GH,True,genderAPI.com,False,0.98,Clara,None
1,2,Dalia Ziada,Dalia,female,female,EG,True,genderAPI.com,False,0.98,Dalia,None
2,3,Fatou Haidara,Fatou,female,female,ML,True,genderAPI.com,False,0.97,Fatou,None
3,7,Diallo Sène,Diallo,female,male,ML,True,genderAPI.com,False,0.86,Diallo,None
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderAPI.com,False,0.98,Fatoumata,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,genderAPI.com,False,0.99,Ícaro,None
3641,4865,Sir Chandler,Sir,male,male,AR,True,genderAPI.com,False,1.0,Sir,None
3642,4866,Julio Fierro,Julio,male,male,CL,True,genderAPI.com,False,1.0,Julio,None
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,genderAPI.com,False,0.75,Wrays,None


In [23]:
result_Wikidata_First_Country_GenderAPIcom['predictedGender'].value_counts()

predictedGender
male       1781
female     1734
unknown      24
ERROR        18
Name: count, dtype: int64

An error appeared when testing the people who comes from South Sudan. Their ISO code (SS) was not listed within GenderAPI.com, which triggered an error. 

For the moment, for ease of use, I take the decision to keep those listing as is, with the ERROR code for some elements. At a later date, those rows may be either retried with a different ISO code (for example, SD, for Sudan) or may be deleted. 

In [33]:
result_Wikidata_First_Country_GenderAPIcom.sort_values(by=['index'])

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
0,0,Clara Benson,Clara,female,female,GH,True,genderAPI.com,False,0.98,Clara,None
1,2,Dalia Ziada,Dalia,female,female,EG,True,genderAPI.com,False,0.98,Dalia,None
2,3,Fatou Haidara,Fatou,female,female,ML,True,genderAPI.com,False,0.97,Fatou,None
3,7,Diallo Sène,Diallo,female,male,ML,True,genderAPI.com,False,0.86,Diallo,None
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderAPI.com,False,0.98,Fatoumata,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,genderAPI.com,False,0.99,Ícaro,None
3641,4865,Sir Chandler,Sir,male,male,AR,True,genderAPI.com,False,1.0,Sir,None
3642,4866,Julio Fierro,Julio,male,male,CL,True,genderAPI.com,False,1.0,Julio,None
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,genderAPI.com,False,0.75,Wrays,None


In [34]:
result_Wikidata_First_Country_GenderAPIcom.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIcom/wikidataFirstCountry.csv", index=False)

### Namsor

In [11]:
result_Wikidata_First_Country_Namsor = await run_batched_wrapper(
    wrapper.NamSorWrapper,
    wikidata_First_Country,
    batch_size=50,
    pause_seconds=10,
    endpoint=NamSorEndpoint.FIRST_NAME_GEO
)
result_Wikidata_First_Country_Namsor


Processing rows 0-50...
Sleeping 13.8s to respect API limits...

Processing rows 50-100...
Sleeping 13.2s to respect API limits...

Processing rows 100-150...
Sleeping 14.5s to respect API limits...

Processing rows 150-200...
Sleeping 14.1s to respect API limits...

Processing rows 200-250...
Sleeping 12.8s to respect API limits...

Processing rows 250-300...
Sleeping 12.5s to respect API limits...

Processing rows 300-350...
Sleeping 13.2s to respect API limits...

Processing rows 350-400...
Sleeping 11.4s to respect API limits...

Processing rows 400-450...
Sleeping 11.9s to respect API limits...

Processing rows 450-500...
Sleeping 13.8s to respect API limits...

Processing rows 500-550...
Sleeping 11.5s to respect API limits...

Processing rows 550-600...
Sleeping 13.3s to respect API limits...

Processing rows 600-650...
Sleeping 12.9s to respect API limits...

Processing rows 650-700...
Sleeping 13.0s to respect API limits...

Processing rows 700-750...
Sleeping 13.3s to respec

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraGenderScale,extraScore,extraProbabilityCalibrated,extraScript
0,0,Clara Benson,Clara,female,female,GH,True,NamSor,0.999099,15.669216,0.999549,LATIN
1,2,Dalia Ziada,Dalia,female,female,EG,True,NamSor,0.998789,15.373957,0.999395,LATIN
2,3,Fatou Haidara,Fatou,female,female,ML,True,NamSor,0.987619,12.789984,0.993810,LATIN
3,7,Diallo Sène,Diallo,female,female,ML,True,NamSor,0.310045,1.876022,0.655023,LATIN
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,NamSor,0.999991,20.272138,0.999995,LATIN
...,...,...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,NamSor,-0.997736,14.748236,0.998868,LATIN
3641,4865,Sir Chandler,Sir,male,male,AR,True,NamSor,-0.964407,9.258472,0.982204,LATIN
3642,4866,Julio Fierro,Julio,male,male,CL,True,NamSor,-0.993507,13.694498,0.996753,LATIN
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,NamSor,-0.228425,1.480006,0.614213,LATIN


In [12]:
result_Wikidata_First_Country_Namsor['predictedGender'].value_counts()

predictedGender
male      1825
female    1820
Name: count, dtype: int64

In [17]:
result_Wikidata_First_Country_Namsor.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Namsor/wikidataFirstCountry.csv", index=False)

### GenderGuesser

In [13]:
genderGuesserHandler = GenderGuesserHandler(wikidata_First_Country)

result_Wikidata_First_GenderGuesser = genderGuesserHandler.get_prediction(useLocalization=False)

In [16]:
result_Wikidata_First_GenderGuesser['extraPreciseGenderPredicted'].value_counts()

extraPreciseGenderPredicted
female           1327
male             1280
unknown           821
andy               90
mostly_male        71
mostly_female      56
Name: count, dtype: int64

In [20]:
result_Wikidata_First_GenderGuesser.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderGuesser/wikidataFirst.csv", index=False)

## Kaggle First + Localization

In [5]:
kaggle_First_Country = master_df[
    (master_df['source'] == 'kaggleOlympic') &
    (master_df['firstName'].notnull()) &
    (master_df['isoCountry'].notnull())
]
kaggle_First_Country.shape

(9078, 11)

### Genderize

In [36]:
result_Kaggle_First_Country_Genderize = await run_batched_wrapper(
    wrapper_class=wrapper.GenderizeWrapper,
    df=kaggle_First_Country,
    batch_size=100,
    pause_seconds=5,
    useLocalization=True
)
result_Kaggle_First_Country_Genderize


Processing rows 0-100...
Sleeping 6.2s to respect API limits...

Processing rows 100-200...
Sleeping 6.7s to respect API limits...

Processing rows 200-300...
Sleeping 8.5s to respect API limits...

Processing rows 300-400...
Sleeping 7.3s to respect API limits...

Processing rows 400-500...
Sleeping 7.8s to respect API limits...

Processing rows 500-600...
Sleeping 9.6s to respect API limits...

Processing rows 600-700...
Sleeping 9.5s to respect API limits...

Processing rows 700-800...
Sleeping 7.5s to respect API limits...

Processing rows 800-900...
Sleeping 7.9s to respect API limits...

Processing rows 900-1000...
Sleeping 7.8s to respect API limits...

Processing rows 1000-1100...
Sleeping 8.4s to respect API limits...

Processing rows 1100-1200...
Sleeping 7.1s to respect API limits...

Processing rows 1200-1300...
Sleeping 8.0s to respect API limits...

Processing rows 1300-1400...
Sleeping 6.3s to respect API limits...

Processing rows 1400-1500...
Sleeping 9.3s to respect 

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
0,4874,ALEKSANYAN Artur,Artur,male,male,AM,True,genderize.IO,138242,1.00
1,4875,AMOYAN Malkhas,Malkhas,male,male,AM,True,genderize.IO,3,1.00
2,4876,GALSTYAN Slavik,Slavik,male,male,AM,True,genderize.IO,2018,0.99
3,4877,HARUTYUNYAN Arsen,Arsen,male,male,AM,True,genderize.IO,10342,0.99
4,4878,TEVANYAN Vazgen,Vazgen,male,male,AM,True,genderize.IO,398,0.98
...,...,...,...,...,...,...,...,...,...,...
9073,15980,HOMAN Khrystyna,Khrystyna,female,female,UA,True,genderize.IO,2032,1.00
9074,15983,LIUZZI Emanuela,Emanuela,female,female,IT,True,genderize.IO,91696,1.00
9075,15984,BOERS Isayah,Isayah,male,male,NL,True,genderize.IO,86,0.97
9076,15985,STAUT Kevin,Kevin,male,male,FR,True,genderize.IO,1257153,1.00


In [37]:
result_Kaggle_First_Country_Genderize.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Genderize/kaggleFirstCountry.csv", index=False)

### GenderAPI.io

In [6]:
result_Kaggle_First_Country_GenderAPIIO = await run_batched_wrapper(
    wrapper_class=wrapper.GenderAPI_IO_Wrapper,
    df=kaggle_First_Country,
    batch_size=200,
    pause_seconds=5,
    useLocalization=True
)
result_Kaggle_First_Country_GenderAPIIO


Processing rows 0-200...
Sleeping 8.6s to respect API limits...

Processing rows 200-400...
Sleeping 7.7s to respect API limits...

Processing rows 400-600...
Sleeping 9.8s to respect API limits...

Processing rows 600-800...
Sleeping 6.3s to respect API limits...

Processing rows 800-1000...
Sleeping 6.4s to respect API limits...

Processing rows 1000-1200...
Sleeping 6.1s to respect API limits...

Processing rows 1200-1400...
Sleeping 8.4s to respect API limits...

Processing rows 1400-1600...
Sleeping 10.0s to respect API limits...

Processing rows 1600-1800...
Sleeping 8.9s to respect API limits...

Processing rows 1800-2000...
Sleeping 7.4s to respect API limits...

Processing rows 2000-2200...
Sleeping 6.0s to respect API limits...

Processing rows 2200-2400...
Sleeping 6.5s to respect API limits...

Processing rows 2400-2600...
Sleeping 8.8s to respect API limits...

Processing rows 2600-2800...
Sleeping 8.0s to respect API limits...

Processing rows 2800-3000...
Sleeping 6.9s 

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraTotalName,extraProbability,extraCountryUsedByService,extraUsedAI,extraForcedGenderize
0,4874,ALEKSANYAN Artur,artur,male,male,AM,True,genderAPI.io,100,100,AM,False,False
1,4875,AMOYAN Malkhas,,male,null,AM,True,genderAPI.io,1,50,AM,False,False
2,4876,GALSTYAN Slavik,slavik,male,male,AM,True,genderAPI.io,346,100,UA,False,False
3,4877,HARUTYUNYAN Arsen,arsen,male,male,AM,True,genderAPI.io,99,99,AM,False,False
4,4878,TEVANYAN Vazgen,vazgen,male,male,AM,True,genderAPI.io,17,100,AM,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9073,15980,HOMAN Khrystyna,khrystyna,female,female,UA,True,genderAPI.io,168,100,UA,False,False
9074,15983,LIUZZI Emanuela,emanuela,female,female,IT,True,genderAPI.io,788,100,IT,False,False
9075,15984,BOERS Isayah,isayah,male,male,NL,True,genderAPI.io,6,100,US,False,False
9076,15985,STAUT Kevin,kevin,male,male,FR,True,genderAPI.io,723,99,FR,False,False


In [9]:
result_Kaggle_First_Country_GenderAPIIO['predictedGender'].value_counts()

predictedGender
male       4730
female     4097
unknown     250
              1
Name: count, dtype: int64

In [15]:
acceptedGender = ['male', 'female', 'unknown']

result_Kaggle_First_Country_GenderAPIIO[~result_Kaggle_First_Country_GenderAPIIO['predictedGender'].isin(acceptedGender)]


,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraTotalName,extraProbability,extraCountryUsedByService,extraUsedAI,extraForcedGenderize
5106,11162,MI Jiujiang,,male,,CN,True,genderAPI.io,1,50,CN,False,False


In [14]:
result_Kaggle_First_Country_GenderAPIIO.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIIO/kaggleFirstCountry.csv", index=False)

The single mistake will be treated "in post". 